In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)




# size of the window for data preparation
split_window_size = 20

# Initialize lists to store training and temporary sets
x_train_list, y_train_list, x_temp_list, y_temp_list = [], [], [], []

# Iterate through the data with the specified window size
for i in range(0, len(x_data) - split_window_size, split_window_size + 1):
    x_train_temp = x_data.iloc[i:i+split_window_size+1]
    y_train_temp = y_data.iloc[i:i+split_window_size+1]

    # Separate the last row for the temporary set
    x_train = x_train_temp.iloc[:-1]
    y_train = y_train_temp.iloc[:-1]

    x_temp = x_train_temp.iloc[-1:]
    y_temp = y_train_temp.iloc[-1:]

    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_temp_list.append(x_temp)
    y_temp_list.append(y_temp)

# Concatenate the lists into pandas DataFrames
x_train = pd.concat(x_train_list)
y_train = pd.concat(y_train_list)
x_temp = pd.concat(x_temp_list)
y_temp = pd.concat(y_temp_list)

# print(y_train.head(50))
x_temp_train, x_temp_val, y_temp_train, y_temp_val = train_test_split(x_temp, y_temp, test_size=0.2, random_state=42)


# Split x_temp and y_temp into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42)


"""
# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))
"""




scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)




"""
# x_train_tensor inverse

x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""



'\n# x_train_tensor inverse\n\nx_test_original = scaler.inverse_transform(x_train_tensor.numpy())\nprint("\nFirst row of x_test_original:")\nprint(x_test_original[0])\n\nprint("\nFirst row of x_train:")\nprint(x_train.head(1))\n\n\n\nprint("\nLast row of x_test_original:")\nprint(x_test_original[-1])\n\nprint("\nLast row of x_train:")\nprint(x_train.tail(1))\n'

In [11]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

In [12]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 8
num_layers = 4

learning_rate = 0.001



num_epochs = 100


model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/100], Training Loss: 20870.95536636071, Validation Loss: 21226.111328125
Epoch [2/100], Training Loss: 17448.162189318446, Validation Loss: 19255.291015625
Epoch [3/100], Training Loss: 14097.051751247065, Validation Loss: 18443.63671875
Epoch [4/100], Training Loss: 11654.700044998128, Validation Loss: 17197.322265625
Epoch [5/100], Training Loss: 9894.788355705781, Validation Loss: 12570.208984375
Epoch [6/100], Training Loss: 8494.775836503071, Validation Loss: 10561.306640625
Epoch [7/100], Training Loss: 7375.439280526989, Validation Loss: 9052.9853515625
Epoch [8/100], Training Loss: 6376.937919079605, Validation Loss: 7670.01416015625
Epoch [9/100], Training Loss: 5492.703210519203, Validation Loss: 6269.61572265625
Epoch [10/100], Training Loss: 4741.847024577456, Validation Loss: 5414.2626953125
Epoch [11/100], Training Loss: 4098.595261514257, Validation Loss: 4724.25732421875
Epoch [12/100], Training Loss: 3525.325186154471, Validation Loss: 4091.63818359375
Epoch [

KeyboardInterrupt: 

In [ ]:
90 min

SyntaxError: invalid syntax (2366398978.py, line 1)

Epoch [50/50], Training Loss: 9.2822550582186, Validation Loss: 33.70623779296875

In [ ]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 16
num_layers = 4

learning_rate = 0.001



num_epochs = 50


model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 17703.91857535056, Validation Loss: 18992.966796875
Epoch [2/50], Training Loss: 14243.053091071744, Validation Loss: 18822.771484375
Epoch [3/50], Training Loss: 10604.081621661904, Validation Loss: 21517.015625
Epoch [4/50], Training Loss: 8126.286914922014, Validation Loss: 15842.8076171875
Epoch [5/50], Training Loss: 6197.057869846636, Validation Loss: 12461.8369140625
Epoch [6/50], Training Loss: 4715.641230768164, Validation Loss: 9259.3134765625
Epoch [7/50], Training Loss: 3609.9183537893477, Validation Loss: 6723.17529296875
Epoch [8/50], Training Loss: 2742.297116577055, Validation Loss: 5135.7705078125
Epoch [9/50], Training Loss: 2129.0141410027877, Validation Loss: 3546.696533203125
Epoch [10/50], Training Loss: 1671.706522727489, Validation Loss: 2327.77734375
Epoch [11/50], Training Loss: 1333.1582644001853, Validation Loss: 1670.9093017578125
Epoch [12/50], Training Loss: 1082.085186289028, Validation Loss: 1292.9425048828125
Epoch [13/50],

In [ ]:
ä

NameError: name 'ä' is not defined

In [ ]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 32
num_layers = 4

learning_rate = 0.001



num_epochs = 50


model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 13414.527682768847, Validation Loss: 19475.103515625
Epoch [2/50], Training Loss: 9373.158118079893, Validation Loss: 23497.64453125
Epoch [3/50], Training Loss: 5576.644881284422, Validation Loss: 13723.880859375
Epoch [4/50], Training Loss: 3221.7287345401587, Validation Loss: 9953.5087890625
Epoch [5/50], Training Loss: 1916.6988242916398, Validation Loss: 7427.533203125
Epoch [6/50], Training Loss: 1229.059090179695, Validation Loss: 4506.482421875
Epoch [7/50], Training Loss: 805.268514297738, Validation Loss: 2701.5810546875
Epoch [8/50], Training Loss: 531.8635023809235, Validation Loss: 1626.02587890625
Epoch [9/50], Training Loss: 380.0427150281917, Validation Loss: 1087.25830078125
Epoch [10/50], Training Loss: 232.58011463454372, Validation Loss: 955.9324340820312
Epoch [11/50], Training Loss: 147.62508014390696, Validation Loss: 812.2144775390625
Epoch [12/50], Training Loss: 102.35466250565631, Validation Loss: 546.8161010742188
Epoch [13/50], 

In [ ]:
ü

NameError: name 'ü' is not defined

In [ ]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 64
num_layers = 4

learning_rate = 0.001



num_epochs = 50


model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 9642.959066813468, Validation Loss: 23937.505859375
Epoch [2/50], Training Loss: 11925.58645333698, Validation Loss: 20238.83984375
Epoch [3/50], Training Loss: 3699.3947514453803, Validation Loss: 26222.625
Epoch [4/50], Training Loss: 1396.4277516190061, Validation Loss: 22949.30859375
Epoch [5/50], Training Loss: 664.3690024763957, Validation Loss: 9220.71484375
Epoch [6/50], Training Loss: 389.85383619119546, Validation Loss: 8889.6884765625
Epoch [7/50], Training Loss: 365.67442031374424, Validation Loss: 12144.5205078125
Epoch [8/50], Training Loss: 401.3654579453064, Validation Loss: 8088.58544921875
Epoch [9/50], Training Loss: 215.66482511694514, Validation Loss: 6389.21923828125
Epoch [10/50], Training Loss: 204.86063985245605, Validation Loss: 4422.1943359375
Epoch [11/50], Training Loss: 261.259700843541, Validation Loss: 3071.5205078125
Epoch [12/50], Training Loss: 57.817470528554296, Validation Loss: 1243.059814453125
Epoch [13/50], Training 

KeyboardInterrupt: 

In [ ]:
import itertools

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [64]
num_layers_list = [ 4]
learning_rates = [0.0001]
window_sizes = [1, 2]
num_epochs = 50

# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

best_validation_loss = float('inf')
best_hyperparameters = None

# Iterate over each hyperparameter combination
for hyperparameters in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparameters

    print()
    print("Hyperparameters:", hyperparameters)

    # Initialize the model with current hyperparameters
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validate the model
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

        avg_train_loss = running_loss / len(x_train_tensor)
        avg_val_loss = val_loss / len(x_val_tensor)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')

    # Update best hyperparameters if validation loss is lower
    if avg_val_loss < best_validation_loss:
        best_validation_loss = avg_val_loss
        best_hyperparameters = hyperparameters

print ()
print("Best Hyperparameters:", best_hyperparameters)
print("Best Validation Loss:", best_validation_loss)



Hyperparameters: (7, 64, 4, 0.0001, 1)


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 22340.40040208146, Validation Loss: 22754.83984375
Epoch [2/50], Training Loss: 19507.356103863578, Validation Loss: 20672.6171875
Epoch [3/50], Training Loss: 16501.656412110755, Validation Loss: 19046.109375
Epoch [4/50], Training Loss: 14218.265664036942, Validation Loss: 17461.986328125
Epoch [5/50], Training Loss: 12436.213578614203, Validation Loss: 14280.5126953125
Epoch [6/50], Training Loss: 10998.713287278058, Validation Loss: 11917.75
Epoch [7/50], Training Loss: 9836.220936741438, Validation Loss: 10688.2119140625
Epoch [8/50], Training Loss: 8901.997328398264, Validation Loss: 9908.849609375
Epoch [9/50], Training Loss: 8019.764883428459, Validation Loss: 13094.5908203125
Epoch [10/50], Training Loss: 7234.46473392383, Validation Loss: 11916.5419921875
Epoch [11/50], Training Loss: 6533.952791569571, Validation Loss: 11234.6181640625
Epoch [12/50], Training Loss: 5888.84207414427, Validation Loss: 10455.419921875
Epoch [13/50], Training Loss: 5

KeyboardInterrupt: 

In [ ]:
ö

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 1

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate

model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 22323.929960377744, Validation Loss: 22742.857421875


# Hyperparameters
learning_rates = [0.00001, 0.000005, 0.000001 ]
window_sizes = [5, 10]
hidden_dims = [256, 512]
n_layers_values = [8, 16]
batch_evaluation_frequencies = [5, 10, 15]

best_params = None

for lr, window_size, hidden_dim, n_layers, batch_freq in product(learning_rates, window_sizes, hidden_dims, n_layers_values, batch_evaluation_frequencies):
    params = {
        'learning_rate': lr,
        'window_size': window_size,
        'hidden_dim': hidden_dim,
        'n_layers': n_layers,
        'batch_evaluation_frequency': batch_freq
    }



(13.03)